In [ ]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf

import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
B = 8
H = 10 # Grid height
W = 10 # Grid width
N = H*W # Nodes in grpah
M = 15 # previous time steps
T = 5 # future time steps
TIME_STEP = 60

In [ ]:
def normalize(vec):
    return (vec-min(vec))/(max(vec)-min(vec))

df = pd.read_csv('gdrive/MyDrive/trace/trace_avg.csv')
MAX_COST = 200
df = df[df['Cost'] < MAX_COST]
df['Cost'] = df['Cost'] / MAX_COST
df['Long'] = normalize(df['Long'])
df['Lat'] = normalize(df['Lat'])

df['TimeStep'] = round(df['TimeSec'] / TIME_STEP)
df['LongStep'] = round(df['Long'] * (H-1))
df['LatStep'] = round(df['Lat'] * (W-1))
df['GraphNode'] = (df['LongStep']-1) * W + df['LatStep']
df = df.groupby(['GraphNode', 'TimeStep'])['Cost'].mean().reset_index()

############################################################

times = np.unique(df['TimeStep'])
dataset = []

for t in times:
    grid = np.ones((H*W, 1)) * -1
    df_t = df[df['TimeStep'] == t]
    for index, row in df_t.iterrows():
        lat_step = row['GraphNode'] % W
        long_step = (row['GraphNode'] - lat_step) / W + 1
        index = int(W*(long_step-1)+lat_step)
        grid[index] = row['Cost']
    dataset.append(grid)
dataset = np.array(dataset)
del df

############################################################

indf = []
outdf = []

for i in range(len(dataset)-M-T-1):
    indf.append(dataset[i:i+M])
    outdf.append(dataset[i+M:i+M+T])

del dataset

L = len(indf)
l = L-L%B
indf = np.swapaxes(np.array(indf), 1, 2)[:l]
outdf = np.swapaxes(np.array(outdf), 1, 2)[:l]

indf = np.reshape(indf, (-1, B, indf.shape[1], indf.shape[2]))
outdf = np.reshape(outdf, (-1, B, outdf.shape[1], outdf.shape[2]))

l = int(0.9*len(indf))
indf_train = indf[:l]
outdf_train = outdf[:l]
indf_test = indf[l:]
outdf_test = outdf[l:]

In [ ]:
class ConvModel(tf.keras.Model):

  def __init__(self, T):
    super(ConvModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(2, 3, padding='same', input_shape=(B,N,M))
    self.conv2 = tf.keras.layers.Conv2D(4, 3, padding='same')
    self.conv3 = tf.keras.layers.Conv2D(2, 3, padding='same')
    self.dense = tf.keras.layers.Dense(T, activation='sigmoid')

  def call(self, x):
    x = tf.reshape(x, (x.shape[0], x.shape[1], x.shape[2], 1))
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = tf.reshape(x, (x.shape[0], x.shape[1], -1))
    x = self.dense(x)
    return x

def loss_function(real, pred, loss_func):
    mask = tf.math.logical_not(tf.math.equal(real, -1.0))
    if loss_func == 'MSE':
        loss = tf.square(tf.cast(real, dtype=tf.float32) - tf.cast(pred, dtype=tf.float32))
    elif loss_func == 'MAE':
        loss = tf.abs(tf.cast(real, dtype=tf.float32) - tf.cast(pred, dtype=tf.float32))
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

In [ ]:
conv = ConvModel(T)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
train_loss = tf.keras.metrics.Mean(name='train_loss')

@tf.function
def train_step(inp, tar):
    
    with tf.GradientTape() as tape:
        pred = conv(inp)
        loss = loss_function(tar, pred, 'MSE')

    gradients = tape.gradient(loss, conv.trainable_variables)
    optimizer.apply_gradients(zip(gradients, conv.trainable_variables))
    train_loss(loss)

In [ ]:
EPOCHS = 10
beginning = time.time()
for epoch in range(EPOCHS):
    for b in range(indf_train.shape[0]):
        inp = indf_train[b]
        tar = outdf_train[b]

        train_step(inp, tar)

        if b % 10 == 0:
            print(f'Epoch {epoch + 1} Batch {b} Loss {train_loss.result():.4f}')
        
    print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f}')
print(f'Total time: {time.time() - beginning:.2f} secs\n')

Epoch 1 Batch 0 Loss 0.1268
Epoch 1 Batch 10 Loss 0.0478
Epoch 1 Batch 20 Loss 0.0298
Epoch 1 Batch 30 Loss 0.0282
Epoch 1 Batch 40 Loss 0.0229
Epoch 1 Batch 50 Loss 0.0235
Epoch 1 Batch 60 Loss 0.0206
Epoch 1 Batch 70 Loss 0.0207
Epoch 1 Batch 80 Loss 0.0211
Epoch 1 Batch 90 Loss 0.0199
Epoch 1 Batch 100 Loss 0.0200
Epoch 1 Batch 110 Loss 0.0187
Epoch 1 Batch 120 Loss 0.0176
Epoch 1 Batch 130 Loss 0.0167
Epoch 1 Loss 0.0166
Epoch 2 Batch 0 Loss 0.0165
Epoch 2 Batch 10 Loss 0.0158
Epoch 2 Batch 20 Loss 0.0149
Epoch 2 Batch 30 Loss 0.0149
Epoch 2 Batch 40 Loss 0.0143
Epoch 2 Batch 50 Loss 0.0146
Epoch 2 Batch 60 Loss 0.0141
Epoch 2 Batch 70 Loss 0.0146
Epoch 2 Batch 80 Loss 0.0147
Epoch 2 Batch 90 Loss 0.0143
Epoch 2 Batch 100 Loss 0.0145
Epoch 2 Batch 110 Loss 0.0141
Epoch 2 Batch 120 Loss 0.0137
Epoch 2 Batch 130 Loss 0.0134
Epoch 2 Loss 0.0133
Epoch 3 Batch 0 Loss 0.0133
Epoch 3 Batch 10 Loss 0.0130
Epoch 3 Batch 20 Loss 0.0127
Epoch 3 Batch 30 Loss 0.0126
Epoch 3 Batch 40 Loss 0.012

In [ ]:
print(np.sum([np.prod(v.get_shape().as_list()) for v in conv.trainable_variables]))
conv.save_weights('gdrive/MyDrive/trace/conv-1')
err = 0.0
cnt = 0

for b in range(indf_test.shape[0]):
    inp = indf_test[b]
    tar = outdf_test[b]
    pred = conv(inp)
    cnt += 1
    err += loss_function(tar, pred, 'MAE')

print(err/cnt)

325
tf.Tensor(0.056200523, shape=(), dtype=float32)
